In [1]:
import layers as layers
import models as models

In [2]:
import numpy as np
import tensorflow as tf
import argparse, sys
sys.path.append("../../")
from utils.generator import ImageAugmentationSequenceH5 # from https://github.com/juselara1/MLSA

In [4]:
#Loading the base xception model

input_tensor = tf.keras.layers.Input(shape=(256, 256, 3))
base_params = {"weights": "imagenet", "input_tensor": input_tensor, "include_top": False, "pooling": "avg"}

preprop = tf.keras.applications.xception.preprocess_input
base = tf.keras.applications.xception.Xception(**base_params)
train_batch = 32
val_batch = 16

In [5]:
#Augmentation set up

augmentation = {"preprocessing_function": preprop, "apply":True,
                    "random_brightness": {"max_delta": 0.3},
                    "random_contrast": {"lower":0.7, "upper":1},
                    "random_hue": {"max_delta": 0.1},
                    "random_saturation": {"lower": 0.7, "upper":1},
                    "random_rotation": {"minval": 0, "maxval": 2*np.pi},
                    "horizontal_flip": True, "vertical_flip": True
                   }


train_seq = ImageAugmentationSequenceH5('/home/stoledoc/work/data1/images.h5', "ims_train", "y_train", num_classes=5,
                                        #labels_transform=lambda i:np.int32(i>1),
                                        #labels_transform=lambda i:np.eye(5)[i],
                                        class_weighted=False, categorical=False, batch_size=train_batch,
                                        #shuffle=True)
                                        augmentation = augmentation, shuffle=True)

augmentation = {"preprocessing_function": preprop, "apply":False}

val_seq = ImageAugmentationSequenceH5('/home/stoledoc/work/data1/images.h5', "ims_val", "y_val", num_classes=5,
                                      #labels_transform=lambda i:np.int32(i>1),
                                      #labels_transform=lambda i:np.eye(5)[i],
                                      class_weighted=False, categorical=False, batch_size=val_batch,
                                      #shuffle=False)
                                      augmentation = augmentation, shuffle=False)


In [6]:
#Model Checkpoint for warming up

callback = tf.keras.callbacks.ModelCheckpoint(filepath='/home/stoledoc/work/datanfs/stoledoc/MICCAI_2021/xception+qmr_warming_up_2.h5', monitor="val_loss", 
                                              verbose=True, save_best_only=True,
                                              save_weights_only=True, mode="min")

In [8]:
#QMR model layers set up

fm_x = layers.QFeatureMapRFF(
                            input_dim=2048,
                            dim=1024, 
                            gamma=0.0006740385616590121, 
                            random_state=1)
qm = layers.QMeasureClassifEig(dim_x=1024, dim_y=5, num_eig=32)

dmregress = layers.DensityMatrixRegression()

dm2dist = layers.DensityMatrix2Dist()

#QMR layers

psi_x = fm_x(base.output)
rho_y = qm(psi_x)
output = dmregress(rho_y)
output_2 = dm2dist(rho_y)

#Final model

#Freezing the layers for warming up:
for layer in base.layers:
    layer.trainable = False

model = tf.keras.Model(inputs=input_tensor, outputs=[output_2, output])

#QMR loss

def loss(y_true, y_pred):
    return tf.keras.losses.mean_squared_error(y_true, y_pred[:,0:1]*4) + 0.5526216196078314 * y_pred[:, 1:2]

model.compile(loss=loss, optimizer=tf.optimizers.Adam(lr=0.0001947103376276921))

#Loading weights from JSLara

model.load_weights('models/xception.h5', by_name = True)

model.fit(train_seq, steps_per_epoch=len(train_seq), validation_data=val_seq, validation_steps=len(val_seq),
          epochs=20, callbacks=[callback], shuffle=True, use_multiprocessing=False)

In [12]:
#UnFreezing the layers after warming up:
for layer in base.layers:
    layer.trainable = True
    
model.compile(loss=loss, optimizer=tf.optimizers.Adam(lr=1e-7))
model.load_weights('/home/stoledoc/work/datanfs/stoledoc/MICCAI_2021/xception+qmr_warming_up_2.h5', by_name = True)

#Model Checkpoint for Full train

callback = tf.keras.callbacks.ModelCheckpoint(filepath='/home/stoledoc/work/datanfs/stoledoc/MICCAI_2021/xception+qmr_full_train_3.h5', monitor="val_loss", 
                                              verbose=True, save_best_only=True,
                                              save_weights_only=True, mode="min")

model.fit(train_seq, steps_per_epoch=len(train_seq), validation_data=val_seq, validation_steps=len(val_seq),
          epochs=20, callbacks=[callback], shuffle=True, use_multiprocessing=False)

# Evaluating

In [48]:
import h5py

file_route = '/home/stoledoc/work/data1/images.h5'

with h5py.File(file_route,'r') as h5:

    
    #X_train = h5['ims_train'].value
    #X_val = h5['ims_val'].value
    X_test = h5['ims_test'].value
    
    #y_train = h5['y_train'].value
    #y_val = h5['y_val'].value
    y_test = h5['y_test'].value
    #id_ims_train = h5['id_train'].value
    #id_ims_val = h5['id_val'].value
    id_ims_test = h5['id_test'].value

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  # Remove the CWD from sys.path while we load stuff.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:14: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:17: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.


In [53]:
#Loading weights 

model.load_weights('/home/stoledoc/work/datanfs/stoledoc/MICCAI_2021/xception+qmr_full_train_3.h5')#, by_name = True)

In [54]:
augmentation = {"preprocessing_function": preprop, "apply":False}

test_seq = ImageAugmentationSequenceH5('/home/stoledoc/work/data1/images.h5', "ims_test", "y_test", num_classes=5,
                                       #labels_transform=lambda i:np.int32(i>1), 
                                       return_y=False,
                                       class_weighted=False, categorical=False, batch_size=20,
                                       augmentation = augmentation, shuffle=False)

In [55]:
out = model.predict(test_seq, steps=len(test_seq), verbose=1)

4387/4387 [==============================] - 460s 105ms/step


In [56]:
out[0].shape, out[1].shape

((87740, 5), (87740, 2))

In [57]:
out[0].shape, out[1].shape

((87740, 5), (87740, 2))

## Patch Level

Para esta evaluaciòn tomamos el modelo que bota solo el mean y el std. Pilas!

In [58]:
from sklearn.metrics import  roc_curve, roc_auc_score, classification_report, confusion_matrix, accuracy_score, mean_absolute_error

In [59]:
y_pred, std = out[1][:, 0], np.sqrt(out[1][:, 1])

In [60]:
y_pred

array([0.6660456 , 0.46914953, 0.6110195 , ..., 0.49020052, 0.27564344,
       0.63827837], dtype=float32)

In [61]:
y_test

array([0, 0, 0, ..., 4, 4, 4], dtype=uint8)

In [62]:
print('------------MAE =', mean_absolute_error(y_test, np.rint(y_pred*4)))

------------MAE = 0.87410533


In [63]:
print(classification_report(y_test, np.rint(y_pred*4), digits = 4))

              precision    recall  f1-score   support

           0     0.1527    0.0997    0.1207      7640
           1     0.6630    0.4249    0.5179     32302
           2     0.2536    0.4742    0.3305     15301
           3     0.4304    0.4707    0.4497     30560
           4     0.0000    0.0000    0.0000      1937

    accuracy                         0.4118     87740
   macro avg     0.3000    0.2939    0.2837     87740
weighted avg     0.4515    0.4118    0.4154     87740



## Image Level: Mayority Vote

In [24]:
y_pred = np.rint(y_pred*4)

In [25]:
keys = list(set(id_ims_test))
keys.sort()
#keys = list(keys)

In [26]:
d_0 = {} 
  
for i in keys: 
    d_0[i] = 0

for i in range(X_test.shape[0]):
  if y_pred[i] == 0:
    d_0[id_ims_test[i]] += 1

d_1 = {} 
  
for i in keys: 
    d_1[i] = 0

for i in range(id_ims_test.shape[0]):
  if y_pred[i] == 1:
    d_1[id_ims_test[i]] += 1

d_2 = {} 
  
for i in keys: 
    d_2[i] = 0

for i in range(id_ims_test.shape[0]):
  if y_pred[i] == 2:
    d_2[id_ims_test[i]] += 1

d_3 = {} 
  
for i in keys: 
    d_3[i] = 0

for i in range(id_ims_test.shape[0]):
  if y_pred[i] == 3:
    d_3[id_ims_test[i]] += 1

d_4 = {} 
  
for i in keys: 
    d_4[i] = 0

for i in range(id_ims_test.shape[0]):
  if y_pred[i] == 4:
    d_4[id_ims_test[i]] += 1

In [27]:
vote_pred = []

for i in range(len(keys)):
  vote = np.argmax([d_0[keys[i]], d_1[keys[i]], d_2[keys[i]], d_3[keys[i]], d_4[keys[i]]])
  vote_pred.append(vote)

vote_pred = np.asarray(vote_pred)

In [28]:
y_test

array([0, 0, 0, ..., 4, 4, 4], dtype=uint8)

In [29]:
d_0 = {} 
  
for i in keys: 
    d_0[i] = 0

for i in range(X_test.shape[0]):
  if y_test[i] == 0:
    d_0[id_ims_test[i]] += 1

d_1 = {} 
  
for i in keys: 
    d_1[i] = 0

for i in range(id_ims_test.shape[0]):
  if y_test[i] == 1:
    d_1[id_ims_test[i]] += 1

d_2 = {} 
  
for i in keys: 
    d_2[i] = 0

for i in range(id_ims_test.shape[0]):
  if y_test[i] == 2:
    d_2[id_ims_test[i]] += 1

d_3 = {} 
  
for i in keys: 
    d_3[i] = 0

for i in range(id_ims_test.shape[0]):
  if y_test[i] == 3:
    d_3[id_ims_test[i]] += 1

d_4 = {} 
  
for i in keys: 
    d_4[i] = 0

for i in range(id_ims_test.shape[0]):
  if y_test[i] == 4:
    d_4[id_ims_test[i]] += 1

In [30]:
vote_test = []

for i in range(len(keys)):
  vote = np.argmax([d_0[keys[i]], d_1[keys[i]], d_2[keys[i]], d_3[keys[i]], d_4[keys[i]]])
  vote_test.append(vote)

vote_test = np.asarray(vote_test)

In [31]:
vote_test

array([0, 1, 3, 1, 1, 3, 3, 1, 1, 1, 1, 1, 1, 3, 3, 3, 1, 1, 1, 1, 1, 1,
       4, 2, 3, 3, 3, 1, 2, 2, 0, 2, 0, 2, 2, 2, 3, 3, 2, 1, 3, 3, 0, 3,
       3, 3])

In [32]:
print('------------MAE =', mean_absolute_error(vote_test, vote_pred))

------------MAE = 0.6304347826086957


In [33]:
print(classification_report(vote_test, vote_pred, digits = 4))

              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000         4
           1     0.9231    0.7059    0.8000        17
           2     0.2778    0.6250    0.3846         8
           3     0.5333    0.5000    0.5161        16
           4     0.0000    0.0000    0.0000         1

    accuracy                         0.5435        46
   macro avg     0.3468    0.3662    0.3401        46
weighted avg     0.5750    0.5435    0.5421        46



/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [34]:
print(confusion_matrix(vote_test, vote_pred))

[[ 0  0  3  1  0]
 [ 0 12  3  2  0]
 [ 0  0  5  3  0]
 [ 0  1  7  8  0]
 [ 0  0  0  1  0]]


## Image Level: Probability Vote

In [35]:
dist_pred = out[0]

In [36]:
dict_dist_pred = {}
for i in keys: 
    dict_dist_pred[i] = [0, 0, 0, 0, 0] # Initializing a null distribution

In [37]:
for i in range(X_test.shape[0]):
    dict_dist_pred[id_ims_test[i]] = dict_dist_pred[id_ims_test[i]] + dist_pred[i]

Just picking the argmax:

In [38]:
prob_pred = []

for i in range(len(keys)):
  prob = np.argmax(dict_dist_pred[keys[i]])
  prob_pred.append(prob)

prob_pred = np.asarray(prob_pred)

In [39]:
prob_pred

array([3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1,
       3, 3, 3, 3, 3, 2, 2, 3, 3, 3, 3, 2, 2, 3, 3, 3, 3, 3, 3, 3, 2, 3,
       3, 3])

In [40]:
print('------------MAE =', mean_absolute_error(vote_test, prob_pred))

------------MAE = 0.717391304347826


In [41]:
print(classification_report(vote_test, prob_pred, digits = 4))

              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000         4
           1     0.7368    0.8235    0.7778        17
           2     0.5000    0.3750    0.4286         8
           3     0.4762    0.6250    0.5405        16
           4     0.0000    0.0000    0.0000         1

    accuracy                         0.5870        46
   macro avg     0.3426    0.3647    0.3494        46
weighted avg     0.5249    0.5870    0.5500        46



/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [42]:
print(confusion_matrix(vote_test, prob_pred))

[[ 0  0  1  3  0]
 [ 0 14  1  2  0]
 [ 0  0  3  5  0]
 [ 0  5  1 10  0]
 [ 0  0  0  1  0]]


Now computing the expected value and setting it as the prediction

In [43]:
labels = [0,1,2,3,4]
E_pred = []
for i in range(len(keys)):
  dist = dict_dist_pred[keys[i]]/np.sum(dict_dist_pred[keys[i]])
  prediction = np.rint(np.dot(dist, labels))
  E_pred.append(prediction)
E_pred = np.asarray(E_pred)

In [44]:
E_pred

array([2., 3., 2., 1., 1., 2., 2., 2., 1., 1., 1., 1., 1., 2., 2., 2., 1.,
       1., 2., 2., 1., 1., 2., 2., 3., 3., 2., 2., 2., 3., 2., 2., 3., 2.,
       2., 2., 3., 2., 3., 3., 2., 3., 2., 3., 3., 3.])

In [45]:
print('------------MAE =', mean_absolute_error(vote_test, E_pred))

------------MAE = 0.6521739130434783


In [46]:
print(classification_report(vote_test, E_pred, digits = 4))

              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000         4
           1     1.0000    0.6471    0.7857        17
           2     0.2609    0.7500    0.3871         8
           3     0.5833    0.4375    0.5000        16
           4     0.0000    0.0000    0.0000         1

    accuracy                         0.5217        46
   macro avg     0.3688    0.3669    0.3346        46
weighted avg     0.6178    0.5217    0.5316        46



/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [47]:
print(confusion_matrix(vote_test, E_pred))

[[ 0  0  3  1  0]
 [ 0 11  4  2  0]
 [ 0  0  6  2  0]
 [ 0  0  9  7  0]
 [ 0  0  1  0  0]]
